In [ ]:
import os
import zipfile
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
# Define the computation hardware approach: GPU if available, else CPU
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

# Load Data

In [ ]:
train_path = "/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip"
test_path = "/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip"

with zipfile.ZipFile(train_path, 'r') as zipp:
    zipp.extractall("data/")
    
with zipfile.ZipFile(test_path, 'r') as zipp:
    zipp.extractall("data/")

In [ ]:
# list of all paths to train images
train_paths = os.listdir('data/train/')
train_paths = ['data/train/' + i for i in train_paths]

# Split train and validation data
train_paths, val_paths = train_test_split(train_paths, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
print(f'Train images: {len(train_paths)}')
print(f'Validation images: {len(val_paths)}')

# Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = Image.open(self.data[idx]).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
            
        if 'dog' in self.data[idx]:
            label = 1
        elif 'cat' in self.data[idx]:
            label = 0
            
        return image, label

### Augmentations & DataLoaders

In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
])

# Create Datasets
train_dataset = CustomDataset(data=train_paths, transform=transform)
val_dataset = CustomDataset(data=val_paths, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

In [ ]:
# Display a photo from the dataset
image = train_dataset.__getitem__(42)[0]
plt.axis('off')
plt.imshow(image.permute(1, 2, 0).cpu().numpy());

# Model

In [ ]:
from torchvision.models import ResNet18_Weights, resnet18

model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(DEVICE)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Training loop

In [ ]:
epochs = 10

for epoch in range(epochs):
    epoch_loss = 0
    epoch_acc = 0
    epoch_loss_val = 0
    epoch_acc_val = 0
    
    for iteration, (X_batch, y_batch) in enumerate(train_loader):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_batch.to(DEVICE))
        
        loss = loss_fn(outputs, torch.nn.functional.one_hot(y_batch.to(DEVICE), 2).float())
        loss.backward()
        optimizer.step()

        acc = ((outputs.argmax(dim=1) == y_batch.to(DEVICE)).float().mean())
        epoch_acc += acc/len(train_loader)
        epoch_loss += loss/len(train_loader)
    
    print(f'Epoch {epoch + 1}: train_loss = {epoch_loss}, train_acc = {epoch_acc}')

    
    for iteration, (X_batch, y_batch) in enumerate(val_loader):  
        model.eval()

        with torch.no_grad():
            outputs = model(X_batch.to(DEVICE))
            loss = loss_fn(outputs, torch.nn.functional.one_hot(y_batch.to(DEVICE), 2).float())
            
            
        # calculate accuracy & loss
        acc = ((outputs.argmax(dim=1) == y_batch.to(DEVICE)).float().mean())
        epoch_acc_val += acc/len(val_loader)
        epoch_loss_val += loss/len(val_loader)
        
    print(f'Epoch {epoch + 1}: val_loss = {epoch_loss_val}, val_acc = {epoch_acc_val}')

In [ ]:
img = train_dataset.__getitem__(22)[0]

model.eval()
with torch.no_grad():
    out = model(img.unsqueeze(0).to(DEVICE))

print(f'Prediction: {"Cat" if out.argmax().item() == 0 else "Dog"}')

plt.axis('off')
plt.imshow(img.permute(1, 2, 0).cpu().numpy());

# Submission

In [ ]:
predictions = np.array([]).astype('int')

for i in range(12500):
    img_path = os.path.join('/kaggle/working/data/test', str(i+1)+'.jpg')
    image = Image.open(img_path).convert("RGB")
    image = transform(image)

    model.eval()
    with torch.no_grad():
        out = model(image.unsqueeze(0).to(DEVICE)).flatten()

    predictions = np.append(predictions, out.argmax().item())

In [ ]:
submission = pd.read_csv('/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')
submission['label'] = predictions
submission.head()

# submission.to_csv('submission.csv')